# Goal

The goal of this assignment is to recreate a table found in the Dodd-Frank Act Stress Test 2021 published by the Federal Reserve.

This table shows hypothetical credit card accounts with variables including Credit Card Type (divided into General Purpose and Private Label), Credit Card Limit (>= 1,500, 1,501 - 7,500, > 7,500) and Product Type (Co-brand and Other).

The table then seperates columns by Lower-risk, Typical and Higher-risk. 

I will recreate this in a dataframe so that the information is able to be indexed and easily selected for future applications.

In [1]:
import pandas as pd
import numpy as np

In [2]:
higher_risk = pd.read_csv('cards-high-risk-2021.csv')
lower_risk = pd.read_csv('cards-low-risk-2021.csv')
typical = pd.read_csv('cards-typical-risk-2021.csv')

# Quick Descriptives

In [3]:
higher_risk.head()

,loan_id,accountoriginationyear,activeflag,borrowerincome,creditcardtype,currentcreditlimit,cycleendingbalance,cycleendingretailapr,dayspastdue,monthendclosedrevokedflag,originalcreditlimit,producttype,refreshedcreditscoreprimaryborrower
0,1,2003,0,0,1,6010,2160,12.11,0,0,1070,2,661
1,2,2004,0,18140,1,530,390,19.16,0,0,870,2,697
2,3,2005,0,0,1,4630,2850,26.44,0,0,260,2,633
3,4,2006,0,98460,1,1490,3490,15.75,0,0,290,2,574
4,5,2006,0,0,1,780,990,18.37,8,0,570,2,693


In [4]:
higher_risk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 13 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   loan_id                              200 non-null    int64  
 1   accountoriginationyear               200 non-null    int64  
 2   activeflag                           200 non-null    int64  
 3   borrowerincome                       200 non-null    int64  
 4   creditcardtype                       200 non-null    int64  
 5   currentcreditlimit                   200 non-null    int64  
 6   cycleendingbalance                   200 non-null    int64  
 7   cycleendingretailapr                 200 non-null    float64
 8   dayspastdue                          200 non-null    int64  
 9   monthendclosedrevokedflag            200 non-null    int64  
 10  originalcreditlimit                  200 non-null    int64  
 11  producttype                     

### Analysis

- We have 12 different columns, none of which have null values and all of which are int64 data type except for "cycleendingratailapr" which is a float

- For our first variable, credit card type, there is a column "creditcardtype" that is directly applicable. This is column index 4

- For our second variable, Credit Card Limit, "currentcreditlimit" can be used. This is column index 5.

- Finally, for product type, there is also a directly applicable column labeled "producttype" at column index 11.

In [5]:
higher_risk['creditcardtype'].value_counts()

1    166
2     34
Name: creditcardtype, dtype: int64

- Credit card type is a binary value of 1 or 2. Based on the example table that we are recreating, it can be interpreted that 1 = General Purpose and 2 = Private Label

In [6]:
higher_risk['currentcreditlimit'].value_counts()

1420    3
930     2
220     2
2670    2
4040    2
       ..
3340    1
8580    1
1920    1
5180    1
8220    1
Name: currentcreditlimit, Length: 177, dtype: int64

- This is a semi-continuous variable

In [7]:
higher_risk['producttype'].value_counts()

2    179
1     21
Name: producttype, dtype: int64

- This is, again, a binary variable with 1 = Co-brand and 2 = Other

# Part 1: Credit Card Types

In [8]:
#This section will give us the percentage of cards
#that are general purpose or private label for the lower risk category
lr_gen_purp = ((lower_risk[lower_risk['creditcardtype'] == 1].sum()) / len(lower_risk['creditcardtype'])) * 100
lr_private = ((lower_risk[lower_risk['creditcardtype'] == 2].count()) / len(lower_risk['creditcardtype'])) * 100

lr_gen_purp = lr_gen_purp['creditcardtype']
lr_private = lr_private['creditcardtype']

#Instantiating the dataframe for credit card type
index = [['Credit Card Type', 'Credit Card Type'],['General Purpose', 'Private Label']]
cols = ['Lower-risk']

cctype_table = pd.DataFrame(data = (lr_gen_purp, lr_private), index = index,
                          columns = cols)

In [9]:
#Finding percentages for general purpose and private label for typical rick profile
typ_gen_purp = ((typical[typical['creditcardtype'] == 1].sum()) / len(typical['creditcardtype'])) * 100
typ_private = ((typical[typical['creditcardtype'] == 2].count()) / len(typical['creditcardtype'])) * 100

typ_gen_purp = typ_gen_purp['creditcardtype']
typ_private = typ_private['creditcardtype']

#adding these to the previous dataframe
cctype_table['Typical'] = [typ_gen_purp, typ_private]

In [10]:
#Finding percentages for gen. purpose and private label for high risk profile
hr_gen_purp = ((higher_risk[higher_risk['creditcardtype'] == 1].sum()) / len(higher_risk['creditcardtype'])) * 100
hr_private = ((higher_risk[higher_risk['creditcardtype'] == 2].count()) / len(higher_risk['creditcardtype'])) * 100

hr_gen_purp = hr_gen_purp['creditcardtype']
hr_private = hr_private['creditcardtype']

#Adding to the dataframe
cctype_table['Higher-risk'] = [hr_gen_purp, hr_private]

#Visualizing dataframe as it now holds all relevant information on credit card types
cctype_table

Lower-risk  Typical  Higher-risk
Credit Card Type General Purpose        88.0     88.5         83.0
                 Private Label          12.0     11.5         17.0

# Part 2: Credit Limit

In [11]:
#creating percentages for credit limits on lower risk
lr_bottom = ((lower_risk[lower_risk['currentcreditlimit'] <= 1500].count()) / 
             len(lower_risk['currentcreditlimit']))*100
lr_mid = ((lower_risk[lower_risk['currentcreditlimit'].between(1500, 7501)].count())/ 
          len(lower_risk['currentcreditlimit']))*100
lr_top = ((lower_risk[lower_risk['currentcreditlimit'] > 7500].count())/len(lower_risk['currentcreditlimit']))*100

#selecting desired columns
lr_bottom = lr_bottom['currentcreditlimit']
lr_mid = lr_mid['currentcreditlimit']
lr_top = lr_top['currentcreditlimit']

#instantiating dataframe for credit limits
index_2 = [['Current Credit Limit', 'Current Credit Limit', 'Current Credit Limit'],
          ['$1,500 and less', '$1,501-7,500', 'Over $7,500']]
cols_2 = ['Lower-risk']

limit_table = pd.DataFrame(data = (lr_bottom, lr_mid, lr_top), index = index_2, columns = cols_2)

In [12]:
#creating percentages for credit limits for typical risk
typ_bottom = ((typical[typical['currentcreditlimit'] <= 1500].count()) /
             len(typical['currentcreditlimit']))*100
typ_mid = ((typical[typical['currentcreditlimit'].between(1500,7501)].count())/
          len(typical['currentcreditlimit']))*100
typ_top = ((typical[typical['currentcreditlimit'] > 7500].count())/len(typical['currentcreditlimit']))*100

#selecting columns
typ_bottom = typ_bottom['currentcreditlimit']
typ_mid = typ_mid['currentcreditlimit']
typ_top = typ_top['currentcreditlimit']

#adding to credit limit dataframe
limit_table['Typical'] = [typ_bottom, typ_mid, typ_top]

In [13]:
#finding percentages for credit limits for higher risk
hr_bottom = ((higher_risk[higher_risk['currentcreditlimit'] <= 1500].count())/
            len(higher_risk['currentcreditlimit']))*100
hr_mid = ((higher_risk[higher_risk['currentcreditlimit'].between(1500,7501)].count())/
         len(higher_risk['currentcreditlimit']))*100
hr_top = ((higher_risk[higher_risk['currentcreditlimit'] > 7500].count())/
         len(higher_risk['currentcreditlimit']))*100

#selecting columns
hr_bottom = hr_bottom['currentcreditlimit']
hr_mid = hr_mid['currentcreditlimit']
hr_top = hr_top['currentcreditlimit']

#finalizing limit_table dataframe
limit_table['Higher-risk'] = [hr_bottom, hr_mid, hr_top]
limit_table

Lower-risk  Typical  Higher-risk
Current Credit Limit $1,500 and less         8.0     16.5         30.0
                     $1,501-7,500           37.5     52.5         60.5
                     Over $7,500            54.5     31.0          9.5

In [14]:
#appending to cctype_table thereby completing the second stage
part_2_table = cctype_table.append(limit_table)

# Part 3: Product Type

In [15]:
#Creating percentages for lower risk product types
lr_co = ((lower_risk[lower_risk['producttype'] == 1].count())/len(lower_risk['producttype']))*100
lr_oth = ((lower_risk[lower_risk['producttype'] == 2].count())/len(lower_risk['producttype']))*100

#selecting columns
lr_co = lr_co['producttype']
lr_oth = lr_oth['producttype']

#instantiating pt_table dataframe
index_3 = [['Product Type', 'Product Type'], ['Co-brand', 'Other']]
cols_3 = ['Lower-risk']

pt_table = pd.DataFrame(data = (lr_co, lr_oth), index = index_3, columns = cols_3)

In [16]:
#finding percentages for typical risk profile product types
typ_co = ((typical[typical['producttype'] == 1].count())/len(typical['producttype']))*100
typ_oth = ((typical[typical['producttype'] == 2].count())/len(typical['producttype']))*100

#selecting columns
typ_co = typ_co['producttype']
typ_oth = typ_oth['producttype']

#adding to dataframe
pt_table['Typical'] = [typ_co, typ_oth]

In [17]:
#finding percentages for higher-risk profile product types
hr_co = ((higher_risk[higher_risk['producttype'] == 1].count())/len(higher_risk['producttype']))*100
hr_oth = ((higher_risk[higher_risk['producttype'] == 2].count())/len(higher_risk['producttype']))*100

#selecting columns
hr_co = hr_co['producttype']
hr_oth = hr_oth['producttype']

#finalizing pt_table dataframe
pt_table['Higher-risk'] = [hr_co, hr_oth]
pt_table

Lower-risk  Typical  Higher-risk
Product Type Co-brand        16.5     15.0         10.5
             Other           83.5     85.0         89.5

# Final Table

In [18]:
#appending to part_2_table in order to create our final table
final_table = part_2_table.append(pt_table)

In [19]:
#setting style to include desired title and only one decimal point per float
table1 = final_table.style.set_caption('Table 1: Summary Statistics of Selected Variables in the' \
                                           ' Portfolios of Hypothetical Credit Card Accounts').format(precision=1)

In [20]:
#visualizing final table
table1